<a href="https://colab.research.google.com/github/jbenasuli/final_project/blob/database/dev/database/Amazon_Vine-PySpark-ETLs/PRELIM_ETL_video_games.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.1.1'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,410 kB]
Hit:12 http://ppa.launchpad.net

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-05-02 22:37:07--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2021-05-02 22:37:07 (11.6 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
import datetime
from pyspark.sql.functions import to_date
from pyspark.sql.functions import col
from pyspark.sql.types import StructField, StringType, IntegerType, StructType, BooleanType, DateType
from pyspark import SparkFiles

### Load Amazon Data into Spark DataFrame

Note: Enter URL for Desired Dataset

In [6]:
#1 load product segment
url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz'
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

### Create DataFrame - Perform Preliminary Cleaning

In [7]:
#1 Check the schema, print row & column count
df.printSchema()
print((df.count(), len(df.columns)))

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)

(1785997, 15)


In [8]:
#2 Drop the round 1 columns
columns_to_drop = ['marketplace', 'product_parent', 'vine', 'review_headline', 'review_body', 'review_date']
df_dropped = df.drop(*columns_to_drop)
df_dropped.show()

+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+-----------------+
|customer_id|     review_id|product_id|       product_title|product_category|star_rating|helpful_votes|total_votes|verified_purchase|
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+-----------------+
|   12039526| RTIS3L2M1F5SM|B001CXYMFS|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|                Y|
|    9636577| R1ZV7R40OLHKD|B00M920ND6|Tonsee 6 buttons ...|     Video Games|          5|            0|          0|                Y|
|    2331478|R3BH071QLH8QMC|B0029CSOD2|Hidden Mysteries:...|     Video Games|          1|            0|          1|                Y|
|   52495923|R127K9NTSXA2YH|B00GOOSV98|GelTabz Performan...|     Video Games|          3|            0|          0|                Y|
|   14533949|R32ZWUXDJPW27Q|B00Y074JOM|Zero Suit Samus a...|  

In [9]:
#3 Filter Step 1:
df_filtered = df_dropped.filter(df_dropped.verified_purchase == 'Y')
df_filtered.show
print((df_filtered.count(), len(df_filtered.columns)))

(1169701, 9)


In [10]:
#4 drop filtered verified_purchase column
columns_to_drop = ['verified_purchase']
df_dropped_2 = df_filtered.drop(*columns_to_drop)
df_dropped_2.show()

+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+
|customer_id|     review_id|product_id|       product_title|product_category|star_rating|helpful_votes|total_votes|
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+
|   12039526| RTIS3L2M1F5SM|B001CXYMFS|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|
|    9636577| R1ZV7R40OLHKD|B00M920ND6|Tonsee 6 buttons ...|     Video Games|          5|            0|          0|
|    2331478|R3BH071QLH8QMC|B0029CSOD2|Hidden Mysteries:...|     Video Games|          1|            0|          1|
|   52495923|R127K9NTSXA2YH|B00GOOSV98|GelTabz Performan...|     Video Games|          3|            0|          0|
|   14533949|R32ZWUXDJPW27Q|B00Y074JOM|Zero Suit Samus a...|     Video Games|          4|            0|          0|
|    2377552|R3AQQ4YUKJWBA6|B002UBI6W6|Psyclone Recharge...|     Video G

## Create Analysis-Specific DFs/Tables
### Perform Analysis-Specific Transforms

### Segmentation Analysis DF

In [11]:
# 1 Create Segmentation DF by Droppig Addtional Columns
segmentation_cols_drop = ['review_id', 'product_id', 'product_title', 'star_rating', 'helpful_votes', 'total_votes']
segmentation_dropped_df = df_dropped_2.drop(*segmentation_cols_drop)
segmentation_dropped_df.show()

+-----------+----------------+
|customer_id|product_category|
+-----------+----------------+
|   12039526|     Video Games|
|    9636577|     Video Games|
|    2331478|     Video Games|
|   52495923|     Video Games|
|   14533949|     Video Games|
|    2377552|     Video Games|
|   17521011|     Video Games|
|   19676307|     Video Games|
|     224068|     Video Games|
|   48467989|     Video Games|
|   48269642|     Video Games|
|   52738710|     Video Games|
|   10556786|     Video Games|
|    2963837|     Video Games|
|   23092109|     Video Games|
|   23091728|     Video Games|
|   10712640|     Video Games|
|   17455376|     Video Games|
|   14754850|     Video Games|
|   14533949|     Video Games|
+-----------+----------------+
only showing top 20 rows



Note: must change Category Label name in withColumnRenamed('count(product_category)', 'name')

In [12]:
#2 Segmentation GroupBy
#2a GroupBy customer_id
#2b Count product_category and rename count columns as Segment Name
segment_df = segmentation_dropped_df.groupby("customer_id")\
.agg({'product_category':'count'}).withColumnRenamed('count(product_category)', 'video_games')
#2c Check results
segment_df.show()

+-----------+-----------+
|customer_id|video_games|
+-----------+-----------+
|   48670265|          1|
|   49103216|          2|
|   46261368|          1|
|    4883305|          5|
|   41192649|          1|
|   10437900|          2|
|   22245671|          1|
|    2574873|          1|
|    4696154|          1|
|    5621202|          1|
|    5871933|          2|
|   44089812|          1|
|    2845910|          1|
|    5274369|          1|
|   39069693|          2|
|     137793|          1|
|   31914942|          3|
|    4707708|          2|
|   17088720|          2|
|   52272486|          3|
+-----------+-----------+
only showing top 20 rows



In [13]:
#3 Check segment_df Schema and Row Count
segment_df.printSchema()
print(segment_df.count())

root
 |-- customer_id: integer (nullable = true)
 |-- video_games: long (nullable = false)

734317


Note: Column Name in df.sort('name' ...) must align with Column name from step 2

In [14]:
#4 Filter for Top n Results
#4a Declare number of rows to filter by (100,000)
row_count = 100000
#4a Sort by Segment Desc and limit to row_count
filtered_segment_df = segment_df.sort('video_games', ascending=False).limit(row_count)
#4b Check Results
filtered_segment_df.show()
print(filtered_segment_df.count())

+-----------+-----------+
|customer_id|video_games|
+-----------+-----------+
|   24594001|        407|
|   10075230|        218|
|    6010757|        170|
|   15913633|        146|
|   12354745|        121|
|    2335562|        114|
|   51310953|        111|
|   15506517|        109|
|   48373154|        105|
|   14886512|        105|
|    5102093|        104|
|   43452218|        103|
|   13548400|        100|
|   42246054|         99|
|   12470583|         94|
|   13516428|         94|
|   44561972|         93|
|   10345078|         92|
|   38614358|         92|
|   14513464|         92|
+-----------+-----------+
only showing top 20 rows

100000


### Segmentation ETL Complete - Add Database Export Code

In [15]:
# Configure settings for RDS
mode = "append"
jdbc_url='jdbc:postgresql://<connection_srtring>'
config = {"user":'<username>', 
          "password": '<password>', 
          "driver":"org.postgresql.Driver"}

Note: table name in table='name_segment' must align with table name in Postgres

In [16]:
# Write segment table to Postgres/RDS
# 54 secs
filtered_segment_df.write.jdbc(url=jdbc_url, table='video_games_segment', mode=mode, properties=config)

## Apriori Analysis DF
### Enter Apriori Transfomations Below - Use df_dropped_2 as Start Point

In [18]:
# 1 Create Apriori DF by Dropping Columns
apriori_cols_drop = ['product_category', 'product_title', 'star_rating', 'helpful_votes', 'total_votes']
apriori_dropped_df = df_dropped_2.drop(*apriori_cols_drop)
apriori_dropped_df.show()

+-----------+--------------+----------+
|customer_id|     review_id|product_id|
+-----------+--------------+----------+
|   12039526| RTIS3L2M1F5SM|B001CXYMFS|
|    9636577| R1ZV7R40OLHKD|B00M920ND6|
|    2331478|R3BH071QLH8QMC|B0029CSOD2|
|   52495923|R127K9NTSXA2YH|B00GOOSV98|
|   14533949|R32ZWUXDJPW27Q|B00Y074JOM|
|    2377552|R3AQQ4YUKJWBA6|B002UBI6W6|
|   17521011|R2F0POU5K6F73F|B008XHCLFO|
|   19676307|R3VNR804HYSMR6|B00BRA9R6A|
|     224068| R3GZTM72WA2QH|B009EPWJLA|
|   48467989| RNQOY62705W1K|B0000AV7GB|
|   48269642|R29DOU8791QZL8|B000A3IA0Y|
|   52738710|R15DUT1VIJ9RJZ|B0053BQN34|
|   10556786|R3IMF2MQ3OU9ZM|B002I0HIMI|
|    2963837|R23H79DHOZTYAU|B0081EH12M|
|   23092109| RIV24EQAIXA4O|B005FMLZQQ|
|   23091728|R3UCNGYDVN24YB|B002BSA388|
|   10712640| RUL4H4XTTN2DY|B00BUSLSAC|
|   17455376|R20JF7Z4DHTNX5|B00KWF38AW|
|   14754850|R2T1AJ5MFI2260|B00BRQJYA8|
|   14533949| RYSSKR5ZTU8JX|B0056WJA6C|
+-----------+--------------+----------+
only showing top 20 rows



### Prelim Apriori ETL Complete - Add Database Export Code

In [19]:
# Configure settings for RDS
mode = "append"
jdbc_url='jdbc:postgresql://<connection_srtring>'
config = {"user":'<username>', 
          "password": '<password>', 
          "driver":"org.postgresql.Driver"}

Note: table name in table='segment_apriori' must align with table name in Postgres

In [20]:
# Write segment_apriori table to RDS
# 2 mins 56 secs
apriori_dropped_df.write.jdbc(url=jdbc_url, table='video_games_apriori', mode=mode, properties=config)

## Amazon Reviews S3 -> Postgres/RDS ETL Complete



### Run Queries in Postgres to Confirm Segment ETL
Check Row Count of Segment Table - SELECT COUNT(*) FROM video_games_segment;

Check 10 Rows of Segment Table - SELECT * FROM video_games_segment LIMIT(10);


In [17]:
# Upon Confirmation of Above Checks Run This Cell
print('Segment ETL Successful')

Segment ETL Successful


### Run Queries in Postgres to Confirm Apriori ETL
Check Row Count of Apriori Table - SELECT COUNT(*) FROM video_games_apriori;

Check 10 Rows of Apriori Table - SELECT * FROM video_games_apriori LIMIT(10);

In [ ]:
# Upon Confirmation of Above Checks Run This Cell
print('Apriori ETL Successful')

## Save Copy to Info
### Path = dev/database/Amazon_Vine-PySpark-ETLs
### Branch = database